In [1]:
import requests 
from bs4 import BeautifulSoup 
import csv 
import pandas as pd 
from lxml import html
import json
import argparse
from collections import OrderedDict
from time import sleep
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)

In [4]:
# List of stock tickers
tickers = ['SHOP','AAPL','TWTR','TSLA','NFLX','FB','DIS','GPRO','SBUX','F','BABA','FIT','GE','FICO','BA','RAD',
           'PG','GME','GILD','RDHL','SNAP','AMD','ATVI','GM','WORK','ZM','DOCU','DPZ','UAL','SNBR','SVMK','ZNGA',
           'BYND','UBER','CMG','AMZN','MMM','SQ','INTC','SFIX','MSFT','PINS','EBAY','PYPL','OSTK','FVRR','HEAR',
           'ETSY','SPOT','VZ','W','LYV','WING','UPWK','PFE','LYFT','HBI','MTCH','TSN','CHGG','GRUB','PLAY','OKTA',
           'WMT','AAL','GPS','CAKE','DAL','AIZ','MS','UAA','MU','NVDA','KO','ORLY','ORCL','BAC','C', 'TWLO','AL','FLIR',
           'COST','FDX','LEVI','LUV','EA','TMUS','UNH','PLUG','PTON','EAT','MRVL', 'AMC','WEN','HTZ','RST','ARCE', 'ADSK',
           'LE','ULTA','TIF','PRTY','GNC','KMX','AZO','SFM','KR','GO','DLTR','DG','BIG','M','KSS','JCP','ZUMZ','AN',
           'URBN','TLYS','BKE','JWN','RL','ROST','LB','EXPR','AEO','BOOT','BURL','FL','DKS','BGFV','ODP','FLWS','HD',
           'LOW','BBY','GIS','ZEN','NTDOY','DKNG','SPY','UA','SNE','GOOGL','LOGI','Z','BB','HTHT', 'ARLO','IFMK','TTWO',
           'BOX','ERI','STMP','TWNK','ROKU','ELY','GOLF','DS','YELP','DBX','SEAS','SNDX','CVNA','YETI','NYT','PLNT',
           'CLW','SHAK','BCRX','SPG','MAC','PEI','TCO','SKT','CBL','REG','CDR','WPG','TME','MRNA','WIX','CAH','GOGO','FCEL',
           'WDAY','CZR','TLRY','GDDY','SCPL','CCL','GLDM','TLT','UUP','GLD','WFC','MVIS','OPK','TAK','WM','CODX','DEO','SPT',
           'LVGO','KC','ZUO','PLUS','AMCX','VERU','PANW','SNX','NVAX','NLOK','INO','HPQ','SNY','SKX','ATHX','ARDS','VIR',
           'VKTX','KPTI','HOG','AZN','SPLK','DEA','YY','SRNE','IOVA','VWAGY','TGLS','APOG','UNFI','CAR','WHHS','CPB','NCLH',
           'XSPA','NICE','NOW','PLAN','T','CREE','CLDR','CRWD','DOW','ESTC','SE','DELL','SIX','CHEF','IZEA','AIMT'
          ]

In [5]:
# Finds any duplicates in tickers list
dictOfElems = dict()
for elem in tickers:
        if elem in dictOfElems:
            dictOfElems[elem] += 1
        else:
            dictOfElems[elem] = 1    
dictOfElems = { key:value for key, value in dictOfElems.items() if value > 1}

dictOfElems

{}

In [6]:
# Create empty dataframes for webscrapping
df3 = pd.DataFrame(columns=['Ticker','Feb 03, 2020','Mar 02, 2020','Apr 01, 2020','May 04, 2020'])
ec = pd.DataFrame(columns=['Symbol','Earnings Date','EPS Estimate','Reported EPS','Surprise(%)'])
stock_profile = pd.DataFrame(columns=['Symbol','Industry','Sector','Employees','RH_Collections'])

In [7]:
for item in tickers:
    if stock_profile['Symbol'].str.contains(item).sum() > 1:
        pass
    else:
        try:
            # Get Robinhood Collections
            url = 'https://robinhood.com/stocks/'+item
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.content, 'html.parser')
            lst = []
            for i in soup.find(text='Collections').findNext().find_all('span'):
                lst.append(i.string)

            # Get stock profile info from Yahoo Finance
            url = "https://finance.yahoo.com/quote/"+item+"/profile?p="+item
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.content, 'html.parser')
            Sector = soup.find(text='Sector').findNext().string
            Industry = soup.find(text='Industry').findNext().string
            Num_Employees = int(soup.find(text="Full Time Employees").findNext().string.replace(',', ''))
            
            # Appends all information gathered to the profile table
            stock_profile = stock_profile.append(
                {
                    "Symbol":item,
                    "Industry":Industry,
                    "Sector":Sector,
                    "Employees":Num_Employees,
                    "RH_Collections":lst
                },
                ignore_index=True
            )
            sleep(1)
        except:
            print('Error with: '+item)

Error with: MU
Error with: JCP
Error with: SPY
Error with: SNE
Error with: STMP
Error with: GDDY
Error with: GLDM
Error with: TLT
Error with: UUP
Error with: GLD
Error with: TAK
Error with: YY
Error with: VWAGY
Error with: WHHS
Error with: ESTC


## % Change

In [8]:
for item in tickers:
    if df3['Ticker'].str.contains(item).sum() > 1:
        pass
    else:
        try:
            url = "https://finance.yahoo.com/quote/"+item+"/history?p="+item
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find_all("table")
            df = pd.read_html(str(table))[0]
            df.dropna(subset=['Volume'],inplace=True)
            df['Change'] = df['Open'].apply(lambda x: (float(df.iloc[0,1]) - float(x))/float(x) )
            df2 = df.pivot_table(columns ='Date', values='Change')[['Feb 03, 2020','Mar 02, 2020','Apr 01, 2020','May 04, 2020','Jun 01, 2020']].reset_index(drop=True)
            df2['Ticker'] = item
            df3 = df3.append(df2)
            sleep(1)
        except:
            print('Error with: '+item)

df3['Average Daily Percent Change'] = ((df3['May 04, 2020'] -df3['Mar 02, 2020']))
df3.rename(columns={'Ticker':'Symbol'},inplace=True)

Error with: KC
Error with: HOG
Error with: WHHS


In [ ]:
df3['Average Daily Percent Change'] = ((df3['May 04, 2020'] -df3['Mar 02, 2020']))

In [ ]:
df3.rename(columns={'Ticker':'Symbol'},inplace=True)

In [ ]:
for item in tickers:
    if ec['Symbol'].str.contains(item).sum() > 1:
        pass
    else:
        try:
            url = 'https://finance.yahoo.com/calendar/earnings?symbol='+item
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find_all("table")
            df = pd.read_html(str(table))[0]
            df['Earnings Date'] = df['Earnings Date'].apply(lambda x: x[:12])
            df['Earnings Date'] = pd.to_datetime(df['Earnings Date'],format="%b %d, %Y")
            df['Year'] = df['Earnings Date'].dt.year
            df['Quarter'] = df['Earnings Date'].dt.quarter
            ec = ec.append(df)
            sleep(1)
        except:
            print("Error on:"+item)
            pass

In [ ]:
print(ec.head(),'/n','/n ', df3.head(), '/n','/n',stock_profile.head())

In [ ]:
ec[(ec['Year']==2020)&(ec['Quarter']==3)].sort_values('Earnings Date').reset_index(drop=True)

In [ ]:
df3.merge(stock_profile, on=('Symbol')).groupby('Industry')['Jun 01, 2020'].mean()

In [21]:
df3[['Symbol','Feb 03, 2020','Mar 02, 2020','Apr 01, 2020','May 04, 2020','Jun 01, 2020','Average Daily Percent Change']].sort_values('Feb 03, 2020')

,Symbol,"Feb 03, 2020","Mar 02, 2020","Apr 01, 2020","May 04, 2020","Jun 01, 2020",Average Daily Percent Change
0,HTZ,-0.779188,-0.729183,-0.397924,0.020528,2.445545,NaN
0,JCP,-0.734208,-0.707197,-0.438571,-0.181250,-0.106412,NaN
0,WPG,-0.701299,-0.677193,0.223404,0.231593,0.393939,NaN
0,CBL,-0.688506,-0.507273,0.231818,0.129167,0.003704,NaN
0,NCLH,-0.664883,-0.510512,0.729524,0.444710,0.143577,NaN
0,GNC,-0.650704,-0.591209,0.582979,0.305263,0.019178,NaN
0,PEI,-0.606599,-0.362140,0.781609,0.435185,0.396396,NaN
0,MAC,-0.510903,-0.463641,1.081439,0.677863,0.609078,NaN
0,EXPR,-0.508642,-0.465054,0.353741,0.064171,0.041885,NaN
0,AIMT,-0.493388,-0.224051,0.318280,0.122026,0.098566,NaN
